In [48]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [15]:
df['Embarked'].value_counts().index[0]

'S'

In [37]:
data = df.copy()

In [38]:
data['Sex'] = data['Sex'].map({'male':0, 'female':1})
rv = np.random.normal(loc=data['Age'].mean(), scale=data['Age'].std(), size=data['Age'].isnull().sum())
rv = np.copysign(rv, 1)
rv_s = pd.Series(rv, index=data[data['Age'].isnull()].index)
data['Age'] = data['Age'].fillna(rv_s)
data['Embarked'] = data['Embarked'].fillna(df['Embarked'].value_counts().index[0])
data['Embarked'] = data['Embarked'].map({'S':0, 'C':1, 'Q':2})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     891 non-null    int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 83.7+ KB


In [33]:
rv.min()

-4.499942269760851

In [36]:
rv = np.copysign(rv, 1)
rv.min()

0.3603270353444614

In [39]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


In [40]:
data.corr()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,1.000000,-0.005007,-0.035144,-0.042939,0.039219,-0.057527,-0.001652,0.012658,-0.030467
Survived,-0.005007,1.000000,-0.338481,0.543351,-0.058600,-0.035322,0.081629,0.257307,0.106811
Pclass,-0.035144,-0.338481,1.000000,-0.131900,-0.287713,0.083081,0.018443,-0.549500,0.045702
Sex,-0.042939,0.543351,-0.131900,1.000000,-0.057958,0.114631,0.245489,0.182333,0.116569
Age,0.039219,-0.058600,-0.287713,-0.057958,1.000000,-0.184160,-0.150109,0.087336,-0.004141
SibSp,-0.057527,-0.035322,0.083081,0.114631,-0.184160,1.000000,0.414838,0.159651,-0.059961
Parch,-0.001652,0.081629,0.018443,0.245489,-0.150109,0.414838,1.000000,0.216225,-0.078665
Fare,0.012658,0.257307,-0.549500,0.182333,0.087336,0.159651,0.216225,1.000000,0.062142
Embarked,-0.030467,0.106811,0.045702,0.116569,-0.004141,-0.059961,-0.078665,0.062142,1.000000


In [42]:
data2 = data.copy()[['Pclass','Sex','Fare', 'Embarked']]
label = data.copy()['Survived']

In [43]:
train_d, test_d, train_l, test_l = train_test_split(data2, label, test_size=0.3)

In [44]:
train_d1, train_d2, train_l1, train_l2 = train_test_split(train_d, train_l, test_size=0.3)

In [47]:
clf1 = SVC()
clf2 = RFC()
clf1.fit(train_d, train_l)
clf2.fit(train_d, train_l)
svm_score = clf1.score(test_d, test_l)
rfc_score = clf2.score(test_d, test_l)
print("[Default parameter]")
print(f"SVM: {svm_score*100:.3f}%\nRFC: {rfc_score*100:.3f}%")

[Default parameter]
SVM: 68.284%
RFC: 83.955%
